In [1]:
import fileinput
import csv

In [2]:
# mouse and rat genes are in the ABC rather than Abc format

def reformat_dbPTM1(): # substrate_phosphoSite still in list format
    #fileOutput = open('dbPTM_prelimOutput1_hmr.gmt', 'w+')
    kinases = ['AKT', 'CAMK1', 'CAMK2','CAMKL', 'CDK', 'CK1', 'CK2', 'DAPK', 'DMPK', 'DYRK', 'GRK', 'GSK', 'MAPK', 'MAPKAPK', 'MLCK', 'NDR', 'PDK1', 'PKA', 'PKC', 'PKD', 'PKG', 'PKN', 'RAD53', 'RSK', 'SGK']
    organisms = ['human', 'mouse', 'rat']
    d = dict((o, dict()) for o in organisms)
    for o in organisms: # has all kinases though, so must post-process to
        for k in kinases:
            d[o][k] = []
    #print(d)
    
    goodOrg = False # if the organism in organisms
    for i in range(0, 25):
        kinase = kinases[i]
        fileInput = fileinput.FileInput('input/dbPTM_{0}_pos.txt'.format(kinase))
        count = 0
        organism = ''
        for line in fileInput:
            if count % 2 == 0:
                organism = line.split('_')[1].lower()
                if organism in organisms:
                    goodOrg = True
                    substrate = line.split('_')[0].replace(">", '')
                    phosphosite = line.split('_')[2].strip('\n')
                else:
                    goodOrg = False
            elif count % 2 == 1:
                if goodOrg:
                    temp = int(len(line) / 2) - 1 # len includes 2 extra chars (\n and \t)
                    newEntry = '{0}_{1}{2}'.format(substrate, line[temp], phosphosite)
                    d[organism][kinase].append(newEntry)
            count = count + 1

    for o in organisms:
        for k in kinases:
            if len(d[o][k]) == 0:
                del d[o][k]
    
    for o in organisms:
        uSubs =  set() # unique substrates (substrates being substrate_phosphosite pair)
        numKSI = 0
        numKins = 0
        allwp = open('output-withphosphosite_dbPTM_all_{0}.gmt'.format(o), 'w+') # w phosphosite
        fwp = open('output-withphosphosite_dbPTM_fourplusinteractions_{0}.gmt'.format(o), 'w+')
        '''allnp = open('output_dbPTM_all_{0}.gmt'.format(o), 'w+')
        fnp = open('output_dbPTM_fourplusinteractions_{0}.gmt'.format(o), 'w+')'''
        wr1 = csv.writer(allwp, delimiter = '\t')
        wr1a = csv.writer(fwp, delimiter = '\t')
        '''wr2 = csv.writer(allnp, delimiter = '\t')
        wr2a = csv.writer(fnp, delimiter = '\t')'''
        for k in d[o]:
            #print(d[o][k]) # substrates (with psites)
            if len(d[o][k]) >= 4: # len(d[o][k]) = # substrates for kinase k
                numKSI += len(d[o][k])
                numKins += 1
                {uSubs.add(x) for x in d[o][k]}
                
                t = list({x.split('_')[0] for x in d[o][k]})                
                d[o][k].insert(0, '{0}_dbPTM_{1}'.format(k, o)) # add kinase to beginning of list
                wr1a.writerow(d[o][k])
                t.insert(0, '{0}_dbPTM_{1}'.format(k, o))
                #wr2a.writerow(t)
                
                # write to files for all interactions
                wr1.writerow(d[o][k])
                #wr2.writerow(t)
                
            else: # write to files for all interactions
                t = list({x.split('_')[0] for x in d[o][k]})
                d[o][k].insert(0, '{0}_dbPTM_{1}'.format(k, o)) # add kinase to beginning of list
                t.insert(0, '{0}_dbPTM_{1}'.format(k, o))
                
                wr1.writerow(d[o][k])
                #wr2.writerow(t)
                        
                
                
        print('{0}\t#kin: {1}\t#ksi: {2}\t#usubs: {3}'.format(o, numKins, numKSI, len(uSubs)))

       

In [3]:
reformat_dbPTM1()
#[withoutPhosphosite1(o) for o in ['human', 'mouse', 'rat']]

human	#kin: 25	#ksi: 5787	#usubs: 4926
mouse	#kin: 24	#ksi: 1518	#usubs: 1342
rat	#kin: 23	#ksi: 1045	#usubs: 918
